In [7]:
import pandas as pd
import numpy as np
import useful
pd.set_option('display.max_rows', None)
pd.set_option('display.max_columns', None)
pd.set_option('display.width', None)

In [8]:
df = pd.read_csv("make.obs.sims.txt", sep="\t")
df2 = pd.read_csv("test_o1.txt", sep="\t")
print(pd.Series(list(set(df2["#POSITIONS"]).intersection(set(df["#POSITIONS"])))))

0       16640000
1       49424385
2       19820544
3       36499463
4        4796425
5       22282249
6       33863696
7       48058384
8       19304466
9       13791251
10      11415572
11       4227094
12      39106585
13      23373852
14      15751197
15      37021724
16        870428
17      47011872
18      46354465
19      35575846
20      43720744
21       5525547
22      15648812
23      45447214
24       3199023
25      38899760
26         86065
27       7151665
28      14835764
29       4003893
30      48588854
31      43978804
32      20705336
33      19394612
34       7635002
35      24576059
36       5179450
37      48496707
38      30777412
39      46385223
40      38731847
41       9637961
42      21952590
43      41967697
44      46305362
45      26392663
46      31078488
47       5902427
48       6641758
49      47779936
50       1058914
51      45242466
52      17428580
53      49303656
54      31860845
55      32163950
56      19394672
57      14481522
58      175494

In [ ]:

with open("samples.txt") as f:
    individuals = [line.strip() for line in f.readlines()]
num_individuals = len(individuals)

print(df)
alt_allele_counts = df.iloc[:, -1].apply(lambda s: s.split().count('1')).values
print(alt_allele_counts[:10])
maf_values = alt_allele_counts / (2 * num_individuals)  # MAF = alt_allele_counts / (2 * N)

df_filtered = df[maf_values >= 0.05]
df_filtered.to_csv("filtered_variants.tsv", sep="\t", index=False)

print(f"Отобрано {len(df_filtered)} вариантов из {len(df)}")

drop_percent = np.random.uniform(0.4, 0.5)
num_to_remove = int(len(df_filtered) * drop_percent)

df_ancient = df_filtered.sample(frac=1 - drop_percent, random_state=42)  # удаляем случайные строки
df_ancient.to_csv("ancient_variants.tsv", sep="\t", index=False)

print(f"Удалено {num_to_remove} вариантов ({drop_percent:.1%})")

In [8]:
! python3 dai.seg.py --obs_samples samples.txt --bed test.bed   --HMM_par par.file.txt --EM no --prepared_file filtered_variants.tsv --o out_filtered.txt --arch_cover arch.covering.chr1.txt --cut_off 0.9

In [12]:
! python3 dai.seg.py --obs_samples samples.txt --bed test.bed --EM_samples 10 --cut_off 0.9 --HMM_par par.file.txt --EM yes --EM_steps 10 --prepared_file make.obs.sims.txt --o out.EM --arch_cover arch.covering.chr1.txt

^C
Traceback (most recent call last):
  File "/home/rusann/Desktop/DAIseg/dai.seg.py", line 188, in <module>
    Lambda_opt = EM_gaps(np.array(SEQ[0 : (args.EM_samples+1)]), Lambda_0, N_st, cover)    
                 ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/home/rusann/Desktop/DAIseg/dai.seg.py", line 172, in EM_gaps
    return EM.EM_algorithm_gaps(P, seq, n_st, MU, RR, lambda_0, epsilon, L, int(args.EM_steps), gaps_numbers, cover )
           ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/home/rusann/Desktop/DAIseg/EM.py", line 215, in EM_algorithm_gaps
    lmbd_new = np.array(E_step_gaps(cut,  p, o_mas, n_states, mu, rr, lmbd,gaps, cover))
                        ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/home/rusann/Desktop/DAIseg/EM.py", line 201, in E_step_gaps
    return new_lambda_i_gaps(O, GAMMA, gaps, cover), new_lambda_n_gaps(O, GAMMA, gap

In [12]:
ND_true_tracts = np.load("ND_true_tracts.npy", allow_pickle= True)

In [27]:
ND_HMM_tracts=useful.read_out('out_filtered.txt.archaic.txt')
N_neanderthal=4
L=1000
N_ref_pop=250
N = 2
n_eu = 20
len_sequence = 5e7

HMM_tracts=[]
REAL_tracts=[]
for idx in range(n_eu):
    HMM_tracts.append([useful.tracts_eu(ND_HMM_tracts[idx], len_sequence ), ND_HMM_tracts[idx]])
    REAL_tracts.append([ useful.tracts_eu(ND_true_tracts[idx], len_sequence), ND_true_tracts[idx]])

df=useful.df_result(REAL_tracts, HMM_tracts, N_neanderthal, L, N_ref_pop, n_eu, N)
print(df.head)

<bound method NDFrame.head of     State     Value      Score  n_eu  n_neand     L  n_ref_pop
0       0  0.996506  precision     0        4  1000        250
1       0  0.999974     recall     0        4  1000        250
2       1  0.998916  precision     0        4  1000        250
3       1  0.870224     recall     0        4  1000        250
4       0  0.996977  precision     1        4  1000        250
5       0  0.999949     recall     1        4  1000        250
6       1  0.998183  precision     1        4  1000        250
7       1  0.902870     recall     1        4  1000        250
8       0  0.995416  precision     2        4  1000        250
9       0  0.999884     recall     2        4  1000        250
10      1  0.994922  precision     2        4  1000        250
11      1  0.831438     recall     2        4  1000        250
12      0  0.995966  precision     3        4  1000        250
13      0  0.999931     recall     3        4  1000        250
14      1  0.998275  prec

In [8]:
ND_HMM_tracts=useful.read_out('out_filtered.EM.txt.modern.txt')
N_neanderthal=4
L=1000
N_ref_pop=250
N = 2
n_eu = 20
len_sequence = 5e7

HMM_tracts=[]
REAL_tracts=[]
for idx in range(n_eu):
    HMM_tracts.append([useful.tracts_eu(ND_HMM_tracts[idx], len_sequence ), ND_HMM_tracts[idx]])
    REAL_tracts.append([ useful.tracts_eu(ND_true_tracts[idx], len_sequence), ND_true_tracts[idx]])

df=useful.df_result(REAL_tracts, HMM_tracts, N_neanderthal, L, N_ref_pop, n_eu, N)
print(df.head)

<bound method NDFrame.head of     State     Value      Score  n_eu  n_neand     L  n_ref_pop
0       0  0.997565  precision     0        4  1000        250
1       0  0.025672     recall     0        4  1000        250
2       1  0.043553  precision     0        4  1000        250
3       1  0.998590     recall     0        4  1000        250
4       0  0.987574  precision     1        4  1000        250
..    ...       ...        ...   ...      ...   ...        ...
75      1  0.998081     recall    18        4  1000        250
76      0  0.995146  precision    19        4  1000        250
77      0  0.021324     recall    19        4  1000        250
78      1  0.039372  precision    19        4  1000        250
79      1  0.997413     recall    19        4  1000        250

[80 rows x 7 columns]>


In [19]:
! python3 dai.seg.py --obs_samples samples.txt --bed test.bed   --HMM_par par.file.txt --EM no --prepared_file ancient_variants.tsv --o out_ancient.txt --arch_cover arch.covering.chr1.txt --cut_off 0.3 --decoding viterbi

In [88]:
! python3 dai.seg.py --obs_samples samples.txt --bed test.bed --EM_samples 10 --cut_off 0.3 --HMM_par par.file.txt --EM yes --EM_steps 10 --prepared_file ancient_variants.tsv --o out_ancient.EM.txt --arch_cover arch.covering.chr1.txt

Число шагов в EM -алгоритме 9


In [14]:
ND_HMM_tracts=useful.read_out('out_ancient.txt.archaic.txt')
N_neanderthal=4
L=2000
N_ref_pop=250
N = 2
n_eu = 20
len_sequence = 5e7

HMM_tracts=[]
REAL_tracts=[]
for idx in range(n_eu):
    HMM_tracts.append([useful.tracts_eu(ND_HMM_tracts[idx], len_sequence ), ND_HMM_tracts[idx]])
    REAL_tracts.append([ useful.tracts_eu(ND_true_tracts[idx], len_sequence), ND_true_tracts[idx]])

df=useful.df_result(REAL_tracts, HMM_tracts, N_neanderthal, L, N_ref_pop, n_eu, N)
print(df["Value"].mean())

0.9687213162500001


In [90]:
ND_HMM_tracts=useful.read_out('out_ancient.EM.txt.archaic.txt')
N_neanderthal=4
L=1000
N_ref_pop=250
N = 2
n_eu = 20
len_sequence = 5e7

HMM_tracts=[]
REAL_tracts=[]
for idx in range(n_eu):
    HMM_tracts.append([useful.tracts_eu(ND_HMM_tracts[idx], len_sequence ), ND_HMM_tracts[idx]])
    REAL_tracts.append([ useful.tracts_eu(ND_true_tracts[idx], len_sequence), ND_true_tracts[idx]])

df=useful.df_result(REAL_tracts, HMM_tracts, N_neanderthal, L, N_ref_pop, n_eu, N)
print(df)

    State     Value      Score  n_eu  n_neand     L  n_ref_pop
0       0  0.997231  precision     0        4  1000        250
1       0  0.999637     recall     0        4  1000        250
2       1  0.985246  precision     0        4  1000        250
3       1  0.897261     recall     0        4  1000        250
4       0  0.997477  precision     1        4  1000        250
5       0  0.999360     recall     1        4  1000        250
6       1  0.978165  precision     1        4  1000        250
7       1  0.919017     recall     1        4  1000        250
8       0  0.996547  precision     2        4  1000        250
9       0  0.999525     recall     2        4  1000        250
10      1  0.980495  precision     2        4  1000        250
11      1  0.873221     recall     2        4  1000        250
12      0  0.996542  precision     3        4  1000        250
13      0  0.999385     recall     3        4  1000        250
14      1  0.985104  precision     3        4  1000    

In [ ]:
! python3 dai.seg.py --obs_samples samples2.txt --bed test.bed  --HMM_par par.file.txt --EM no --prepared_file observs_spain.txt --o test--arch_cover preparations/hg19.arch.covering.chr22.txt --cut_off 0.65 --decoding viterbi/posterior